In [1]:
import numpy as np
import pandas as pd
from utils import get_data, get_preds, get_table, plot_rmse, other_stats, add_intervals_to_test, plot_interval_check, plot_interval_sizes
np.random.seed(2024)

save_val = False
train_yr, test_yr = [2022], [2023]
train_bos, test_bos = get_data(racename="bos", size_train=1000, size_test=1000, train_lis=train_yr, test_lis=test_yr, save=save_val)
train_nyc, test_nyc = get_data(racename="nyc", size_train=1000, size_test=1000, train_lis=train_yr, test_lis=test_yr, save=save_val)
train_chi, test_chi = get_data(racename="chi", size_train=1000, size_test=1000, train_lis=train_yr, test_lis=test_yr, save=save_val)
data = {"bos": (train_bos, test_bos), "nyc": (train_nyc, test_nyc), "chi": (train_chi, test_chi)}
# test_nyc = pd.read_csv("processed_data/test_nyc.csv")

In [2]:
race = "chi"
train, test = data[race]

model_info = [
    # ("rstan2d", "stan_results/params_bos0d.csv", ["total_pace", "curr_pace", "prop"], ["beta[1]", "beta[2]", "beta[3]"], True),
    ("model1", f"stan_results/model1/params_{race}.csv", ['total_pace', 'prop'], ['beta.1', 'beta.2'], True),
    ("model2", f"stan_results/model2/params_{race}.csv", ['total_pace', 'curr_pace', 'prop'], ['beta.1', 'beta.2', 'beta.3'], True),
]

# models = {name: pd.read_csv(path) for (name, path, feats, betas, pleft) in model_info}
mpreds = {name: get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name)
           for (name, path, feats, betas, pleft) in model_info}

test2 = get_table(test, mpreds)
test2

,id,dist,curr_pace,total_pace,finish,age,gender,year,prop,propleft,male,propxcurr,malexage,extrap,model1,model2
0,52098,5K,2.530364,2.530364,1.988080,-,-,2023,0.118497,0.881503,0,0.299842,,-75.808933,-48.290676,-47.899880
1,48876,5K,3.424658,3.424658,3.372632,-,-,2023,0.118497,0.881503,0,0.405813,,-3.167667,11.700534,11.983237
2,55554,5K,2.313744,2.313744,2.334052,-,-,2023,0.118497,0.881503,0,0.274173,,2.644650,35.737519,36.168202
3,86994,5K,2.600104,2.600104,2.817696,-,-,2023,0.118497,0.881503,0,0.308106,,20.886617,46.911149,47.290631
4,75453,5K,3.581662,3.581662,3.368324,-,-,2023,0.118497,0.881503,0,0.424418,,-12.435933,1.150082,1.419692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,48564,40K,3.748126,3.938946,3.932432,-,-,2023,0.947980,0.052020,0,3.553147,,-0.295740,0.302017,0.284222
7996,55791,40K,2.754821,2.815910,2.819956,-,-,2023,0.947980,0.052020,0,2.611514,,0.358323,1.164538,0.537974
7997,81247,40K,2.385496,2.654104,2.645123,-,-,2023,0.947980,0.052020,0,2.261402,,-0.899648,-0.051073,0.056710
7998,46775,40K,3.160556,3.084040,3.091209,-,-,2023,0.947980,0.052020,0,2.996143,,0.528813,1.273269,0.368405


In [3]:
# s_data =pd.DataFrame([[4, 0.2, .5], [3, 0.5, .75]], columns=["total_pace", "prop", "propleft"])
# get_preds(s_data, models['rstan2c'], feats_lis = ["total_pace", "prop"], beta_lis = ["beta[1]", "beta[2]"], full=True)

In [4]:
labels = ["model1", "model2", "extrap"]
rmse_table = plot_rmse(test2, labels, save_name=f"{race}")
rmse_table["pcnt_model1"] = 1 - (rmse_table["model1"] / rmse_table["extrap"])
rmse_table["pcnt_model2"] = 1 - (rmse_table["model2"] / rmse_table["extrap"])
rmse_table

File saved: analysis/chi_rmse.png


,model1,model2,extrap,pcnt_model1,pcnt_model2
dist,,,,,
5K,22.164877,22.261884,28.449815,0.220913,0.217503
10K,19.515898,19.552914,24.781586,0.212484,0.210990
15K,16.615783,16.474499,23.166281,0.282760,0.288859
20K,14.210665,14.013442,20.419777,0.304073,0.313732
25K,11.521237,11.226055,16.455288,0.299846,0.317784
30K,8.444873,8.053165,11.774032,0.282754,0.316023
35K,5.466757,5.050119,7.146656,0.235061,0.293359
40K,2.177568,1.915523,2.254476,0.034114,0.150347


In [5]:
other_stats(test2[labels], test2["finish"])

,model1,model2,extrap
0,14.092823,13.989926,18.850479
1,0.946113,0.946897,0.903587


In [6]:
mpreds2 = {name: (42195 / 60) / get_preds(test, pd.read_csv(path), feats_lis=feats, beta_lis=betas, propleft=pleft, name=name, full=True)
           for (name, path, feats, betas, pleft) in model_info}
# model_preds2["new1"] = (42195 / 60) / get_preds(test, pd.read_csv("stan_results/params_bos1.csv"), feats_lis=['total_pace', 'curr_pace', 'prop'], beta_lis=['beta.1', 'beta.2', 'beta.3'], propleft=True, full=True)

In [7]:
pred_names = labels[:2]
intervals_tbl = add_intervals_to_test(test2, mpreds2, pred_names)
i_check = plot_interval_check(intervals_tbl, pred_names, save_name=f"{race}")
i_sizes = plot_interval_sizes(intervals_tbl, pred_names, save_name=f"{race}")

File saved: analysis/chi_interval_check.png
File saved: analysis/chi_interval_sizes.png


<Figure size 4000x4000 with 0 Axes>

<Figure size 3000x2000 with 0 Axes>